# Interim Notebook (Labs 4 + 5)
### Owen Yaggy
### 10/13/23
### MUSC H255

## Importing libraries:
All the libraries that could be needed. This is directly from the Encoding Music tutorial on Spotify functions on GitHub. As a result, not all of these libraries are currently being used. In addition, I was having issues with the community library, but since it isn't currently necessary, I removed that import statement for the time being.

Finally, I added my own input statement, `import config`. On my local machine, I have a file called config.py. Since I store all my code for this class in a GitHub repository, I want to avoid any unnecessary personal data in the code. My config.py file is on my .gitignore list, so my credentials stay private.

Unusual dependencies:
* Download spotify_tools.py from the Encoding Music GitHub
* A file, config.py, with variables for `client_id`, `client_secret`, and `user` stored as strings. These correspond to your public and private Spotify API keys, and your Spotify username, respectively, OR:
* Replace the usage of `config.client_id` with your private API key, and so on for the other variables. They are only used once, in the code block following the import statements code block.

In [42]:
import pandas as pd
import numpy as np
import random
import altair as alt
import plotly.graph_objects as go
import requests
import inspect
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotify_tools
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import plotly.express as px
import pyvis
from pyvis import network as net
from itertools import combinations
# from community import community_louvain
from copy import deepcopy
import time

import config   # A file I store on my local device containing my Spotify API credentials

Store the credentials based on the config.py and set up the spotipy library with those credentials.

**This is where you can replace the variable usage with your own API keys and username.** 

In [43]:
# storing the credentials:
#
CLIENT_ID = config.client_id
CLIENT_SECRET = config.client_secret
my_username = config.user

# instantiating the client.  This 'sp' version of the client is used repeatedly below
# source: Max Hilsdorf (https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6)
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Take a specific playlist, store it in a dataframe, and show the result.

In [44]:
creator_id = "lzsfootball"
playlist_id = "1hTUFxr380FuNNYHpynyzy"

# playlist_tracks(user_id: String, playlist_id: String): json_dict
playlist_tracks = pd.DataFrame(sp.user_playlist_tracks(creator_id, playlist_id))
playlist_tracks

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:42:39Z', 'added_by...",100,None,0,None,10
1,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:42:42Z', 'added_by...",100,None,0,None,10
2,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:43:22Z', 'added_by...",100,None,0,None,10
3,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:44:56Z', 'added_by...",100,None,0,None,10
4,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:45:31Z', 'added_by...",100,None,0,None,10
5,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:46:20Z', 'added_by...",100,None,0,None,10
6,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:47:14Z', 'added_by...",100,None,0,None,10
7,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:48:01Z', 'added_by...",100,None,0,None,10
8,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:50:00Z', 'added_by...",100,None,0,None,10
9,https://api.spotify.com/v1/playlists/1hTUFxr38...,"{'added_at': '2023-10-06T17:51:32Z', 'added_by...",100,None,0,None,10


Experiment: show a single song's name.
*Challenge:* Because the data returned by Spotify's API is so voluminous and complex, it's actually quite difficult to find out how to query a very specific piece of data.

In [45]:
sample_track = playlist_tracks.iloc[0]["items"]["track"]['name']
sample_track

'Dumb'

Show what one row in data looks like.

In [46]:
playlist_tracks.iloc[0]['items']

{'added_at': '2023-10-06T17:42:39Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/lzsfootball'},
  'href': 'https://api.spotify.com/v1/users/lzsfootball',
  'id': 'lzsfootball',
  'type': 'user',
  'uri': 'spotify:user:lzsfootball'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6olE6TJLqED3rqDCT0FyPh'},
     'href': 'https://api.spotify.com/v1/artists/6olE6TJLqED3rqDCT0FyPh',
     'id': '6olE6TJLqED3rqDCT0FyPh',
     'name': 'Nirvana',
     'type': 'artist',
     'uri': 'spotify:artist:6olE6TJLqED3rqDCT0FyPh'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
 

**First function:** Given the dataframe of tracks, return the Spotify song IDs.
It seems like this wasn't truly necessary, after discovering the function I was trying to build with this data already existed on the Encoding Music GitHub, but it was a useful experiment to try and get specific information from the dataframe.
However, this can now be used to query the results of the `analyze_playlist` function based on specific song ids.

In [58]:
def get_song_ids(tracks: pd.DataFrame) -> dict:     # Parameter is a dataframe, output is a dict
    """Returns the Spotify song IDs contained in the playlist dataframe, matched with the song titles to make them easy to identify"""
    song_ids = {}       # Empty dict to contain all outputted song ids
    for track in tracks['items']:           # Iterate through tracks in dataframe
        song_ids[track['track']['name']] = track['track']['id']          # add song id to list
    return song_ids    # return song ids

print(get_song_ids(playlist_tracks))

{'Dumb': '5gRcv46AMTrosmTOqrOV3Q', 'Creep': '70LcF31zb1H0PyJoS1Sx1r', 'Jerk': '065bY5WrLvI3NN9VCK0QLe', 'Son of a Bitch': '6Al4oITGudbYvcBzO65ah6', 'Dickhead': '25aNJcqUnZmEKBmPhW9I82', 'Snitch': '3sXaTInkrXe5YEaa6ZYr1S', 'Moron': '3OmhbyRDJDm1zhfVEV7d0w', 'Heartbreaker': '22ML0MuFKfw16WejbxsLOy', 'Jackass': '2UiUTpQkTCmE1OX6senl6j', 'Fucker': '2khc6kEG4eBvYN1GvR6R97'}


**Get music feature data!** This is needed to analyze the songs and determine similarity.

*This code is taken from the EncodingMusic GitHub, in spotify_tools.py, but MODIFIED as follows:*

* Previously, the index of every song in the dataframe was sequential (0, 1, 2, etc.)
* This made it hard for me to query a specific audio feature for a track
* So, I changed the code so that it the index becomes the track_id for a song
* This means you can query based on track_id, which makes it easier to find specific data for a song
* HOWEVER, I did not consider that playlists could contain several copies of the exact same song
* I believe this now might pose errors if creating a dataframe for a playlist that contains the same song more than once
* I'm not sure how I would want to resolve that

**Problems Encountered**

Ultimately, the difficult part of this lab has been figuring out how to work with Pandas. While ultimately the tasks are not overly complex, there are so many ways of retrieving data that the syntax for doing so is rather convoluted. I've encountered countless error messages so far related to querying data in the wrong way, which I did forget to record as I was encountering them. 

In [93]:
def analyze_playlist(creator, playlist_id, spotipy_client):
    playlist_features_list = ["artist", "album", "track_name",
                              "danceability", "energy", "key", "loudness", "mode", "speechiness",
                              "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"] # removed track_id
    playlist_df = pd.DataFrame(columns=playlist_features_list)

    playlist_features = {}

    playlist = spotipy_client.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]

        audio_features = spotipy_client.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[3:]: # changed from 4 to 3 to fix error caused by removing track_id from list of features
            playlist_features[feature] = audio_features[feature]

        track_df = pd.DataFrame(playlist_features, index=[playlist_features['track_id']]) # changed index from automatically being 0 to being the track_id
        playlist_df = pd.concat([playlist_df, track_df], ignore_index=False) # changed ignore_index from True to False

    return playlist_df

I needed to test what how I could query the data returned from this function:

In [101]:
df = analyze_playlist(creator_id, playlist_id, sp)
print(df)
print(df.at['5gRcv46AMTrosmTOqrOV3Q', 'liveness'])
print(type(df.at['5gRcv46AMTrosmTOqrOV3Q', 'liveness']))
x = df.loc['5gRcv46AMTrosmTOqrOV3Q']
got = x.get('liveness')
print(got)

/var/folders/24/qxr9c06j19gcvdhrn3t51m000000gn/T/ipykernel_78648/3211627563.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  playlist_df = pd.concat([playlist_df, track_df], ignore_index=False) # changed ignore_index from True to False


0.173


In [102]:
def similarity(id1: str, id2: str, playlist_df: pd.DataFrame) -> None:
    """Returns a scalar from 0-1 that tells you how similar 2 songs are based on a mash-up of their audio features."""
    similarities = {}
    
    # A list of features to compare:
    features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence', 'loudness', 'tempo', 'mode', 'key']
    
    track1 = df.loc[id1] # variable containing all feature data from track 1
    track2 = df.loc[id2] # variable containing all feature data from track 2
    
    # Store all music feature data in dictionary form:
    track1_features = {}
    track2_features = {}
    for feature in features:
        track1_features[feature] = track1.get(feature)
        track2_features[feature] = track2.get(feature)
    
    feature_similarities = {}
    for feature in features:
        if feature not in ['loudness', 'tempo', 'mode', 'key']:# these features aren't stored as scalars, meaning their comparisons need to be handled differently
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature])
        if feature == 'loudness':
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature]) * (1/60)  # loudness is stored -60 to 0, this converts the difference to a positive scalar
            # In rare cases, loudness falls outside the -60 to 0 range. If so, the code to scale down the difference between loudness to a scalar might result in a value over 1.
            # This code caps the maximum amount of difference at 1 - in other words, tracks that have over 60 dB of loudness difference are treated as having 60 dB of loudness difference.
            if feature_similarities[feature] > 1:
                feature_similarities[feature] = 1
        if feature == 'tempo':
            # Tempo is measured in BPM. This could be any positive integer, but most are within a range of 40-200 BPM.
            # For the purpose of comparison, tracks under 40 BPM are treated as having a BPM of 40. Tracks over 200 are treated as having a BPM of 200.
            # Thus, the maximum difference is 160, and the minimum is 0.
            # So to convert to a scalar, the difference between BPM is divided by 160.
            if track1_features[feature] < 40:
                track1_features[feature] = 40
            if track1_features[feature] > 200:
                track1_features[feature] = 200
            if track2_features[feature] < 40:
                track2_features[feature] = 40
            if track2_features[feature] > 200:
                track2_features[feature] = 200
            
            feature_similarities[feature] = abs(track1_features[feature] - track2_features[feature]) * (1/140)
        if feature
    
    return None

song_ids = get_song_ids(playlist_tracks)
similarity(song_ids['Dumb'], song_ids['Creep'], playlist_tracks)

## Things I still want to explore:
* Visualizing data that is collected
* Using numpy to analyze data
* Merging dataframes with Pandas, and the other powers of Pandas
* What are Louvain communities, and how can they be used?
* Using the NetworkX library